In [29]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import pickle 
from scipy.interpolate import UnivariateSpline
from scipy.signal import find_peaks_cwt

from skimage.filters import threshold_minimum
from skimage.transform import resize

os.environ['KERAS_BACKEND'] = 'tensorflow'
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import keras.losses
import tensorflow as tf

from scipy.ndimage.measurements import label
import time


# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
pi = 3.14159

%matplotlib inline

In [30]:
IMG_HEIGHT = 960
IMG_WIDTH = 1280
IMG_CHANNELS = 3
smooth = 1.

In [31]:
heatmap_10 = [np.zeros((IMG_HEIGHT, IMG_WIDTH))]*10


def smooth_heatmap(heatmap):
    # Smoothing heatmap as average of 10 previous frames
    global heatmap_10
    
    heatmap_10_1 = heatmap_10[1:]
    heatmap_10_1.append(heatmap)
    
    heatmap_10 = heatmap_10_1
    
    heatmap = np.mean(heatmap_10,axis=0)
    
    return heatmap 

In [32]:
def IOU_calc(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)

    return (2 * (intersection + 1)) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)


def IOU_calc_loss(y_true, y_pred):
    return 1 - IOU_calc(y_true, y_pred)

In [33]:
# Build a U-Net model
def unet_model_builder():
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = Lambda(lambda x: x / 255) (inputs)

    c1 = BatchNormalization() (inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same') (c1)
    c1 = BatchNormalization() (c1)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = BatchNormalization() (p1)
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same') (c2)
    c2 = BatchNormalization() (c2)
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = BatchNormalization() (p2)
    c3 = Conv2D(64, (3, 3), activation='relu', padding='same') (c3)
    c3 = BatchNormalization() (c3)
    c3 = Conv2D(64, (3, 3), activation='relu', padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = BatchNormalization() (p3)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same') (c4)
    c4 = BatchNormalization() (c4)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same') (c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = BatchNormalization() (p4)
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same') (c5)
    c5 = BatchNormalization() (c5)
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same') (c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = BatchNormalization() (u6)
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same') (c6)
    c6 = BatchNormalization() (c6)
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same') (c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = BatchNormalization() (u7)
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same') (c7)
    c7 = BatchNormalization() (c7)
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same') (c7)

    u8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = BatchNormalization() (u8)
    c8 = Conv2D(64, (3, 3), activation='relu', padding='same') (c8)
    c8 = BatchNormalization() (c8)
    c8 = Conv2D(64, (3, 3), activation='relu', padding='same') (c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = BatchNormalization() (u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same') (c9)
    c9 = BatchNormalization() (c9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same') (c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

In [34]:
keras.losses.IOU_calc_loss = IOU_calc_loss

model = unet_model_builder()
model.compile(optimizer='adam', loss='IOU_calc_loss', metrics=[IOU_calc])
model.load_weights('model/model-aircraft-detection(IOU).h5')  

In [35]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 960, 1280, 3) 0                                            
__________________________________________________________________________________________________
batch_normalization_37 (BatchNo (None, 960, 1280, 3) 12          input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (None, 960, 1280, 64 1792        batch_normalization_37[0][0]     
__________________________________________________________________________________________________
batch_normalization_38 (BatchNo (None, 960, 1280, 64 256         conv2d_39[0][0]                  
__________________________________________________________________________________________________
conv2d_40 

In [36]:
def draw_labeled_bboxes(img, labels):
    # Iterate through all detected aircrafts
    for aircraft_number in range(1, labels[1]+1):
        # Find pixels with each aircraft_number label value
        nonzero = (labels[0] == aircraft_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        if (np.max(nonzeroy)-np.min(nonzeroy)) * (np.max(nonzerox)-np.min(nonzerox)) >= 2500:
            bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
            # Draw the box on the image
            print(bbox)
            cv2.rectangle(img, bbox[0], bbox[1], (0,255,0), 3)
    # Return the image
    return img

In [37]:
def test_new_img(img):
    # Test Unet on new image
    img = resize(img,(IMG_HEIGHT, IMG_WIDTH), mode='constant')
    img = np.reshape(img,(1,IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    pred = model.predict(img)
#     pred = (pred > 0.9).astype(int)
    return pred,img[0]

In [38]:
def get_BB_new_img(img):
    
    # Take in RGB image
    pred,img = test_new_img(img)
    img  = np.array(255*img,dtype= np.uint8)
    img_pred = np.array(255*pred[0],dtype=np.uint8)
    heatmap = img_pred[:,:,0]
    heatmap = smooth_heatmap(heatmap)
    heatmap = (heatmap >= 0.6*255).astype(int)
    
    # Use Scipy label function to segregate each individual aircraft in the predicted mask
    labels = label(heatmap) # returns a tuple of (labeled array, number of features)
    draw_img = draw_labeled_bboxes(np.copy(img), labels)
    return draw_img

In [39]:
def get_Unet_mask(img):
    # Take in RGB image
    pred,img = test_new_img(img)
    img  = np.array(255*img,dtype= np.uint8)
    img_pred = np.array(255*pred[0],dtype=np.uint8)
    heatmap = img_pred[:,:,0]
    heatmap = smooth_heatmap(heatmap)
    labels = label(heatmap)
    return heatmap

In [40]:
def pipeline_process_airport(image):
    result = get_BB_new_img(image)
    return result

In [41]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
# clip = VideoFileClip("video/KLM_taxiing.mp4");
# white_clip = clip.subclip(15,90)
# %time white_clip.write_videofile(short, audio=False);

In [ ]:
# Detect aircrafts in london_short.mp4 video
demo_london_path = 'video/london_short_demo.mp4'
clip_london = VideoFileClip('video/london_short.mp4');
demo_london = clip_london.fl_image(pipeline_process_airport)
%time demo_london.write_videofile(demo_london_path, audio=False);

In [ ]:
# Detect aircrafts in KLM_taxiing_short.mp4 video
demo_klm_path = 'video/KLM_taxiing_short_demo.mp4'
clip_klm = VideoFileClip('video/KLM_taxiing_short.mp4');
demo_klm = clip_klm.fl_image(pipeline_process_airport)
%time demo_klm.write_videofile(demo_klm_path, audio=False);

In [ ]:
# Detect aircrafts in eva.mp4 video
demo_eva_path = 'video/eva_demo.mp4'
clip_eva = VideoFileClip('video/eva.mp4');
clip_eva = clip_eva.subclip(120, 200)
demo_eva = clip_eva.fl_image(pipeline_process_airport)
%time demo_eva.write_videofile(demo_eva_path, audio=False);

[MoviePy] >>>> Building video video/eva_demo.mp4
[MoviePy] Writing video video/eva_demo.mp4




  0%|          | 0/2398 [00:00<?, ?it/s]

  0%|          | 1/2398 [00:02<1:21:42,  2.05s/it]

  0%|          | 2/2398 [00:04<1:23:19,  2.09s/it]

  0%|          | 3/2398 [00:06<1:24:45,  2.12s/it]

  0%|          | 4/2398 [00:08<1:24:43,  2.12s/it]

((784, 486), (839, 545))
((342, 502), (467, 569))
((616, 552), (721, 576))
((258, 819), (329, 863))




  0%|          | 5/2398 [00:10<1:28:11,  2.21s/it]

((782, 486), (855, 546))
((313, 501), (497, 570))
((611, 551), (723, 577))
((206, 818), (361, 869))


...



 27%|██▋       | 646/2398 [28:10<1:13:46,  2.53s/it]

((686, 442), (1240, 578))
((515, 517), (581, 556))
((191, 521), (279, 552))
((253, 780), (377, 901))
((994, 828), (1067, 893))




 27%|██▋       | 647/2398 [28:13<1:14:16,  2.55s/it]

((686, 442), (1239, 578))
((190, 520), (279, 552))
((250, 780), (377, 901))
((994, 828), (1067, 893))




 27%|██▋       | 648/2398 [28:15<1:13:15,  2.51s/it]